# Triton (Quad-Apollo) clock configuration
This example models the clock configuration for the Triton (Quad-Apollo) platform. Not all 4 AD9084s are modeled since they share the same clock and JESD configuration. The main limitation compared to the AD9084 FMC single chip board is the reduction of lanes, since all 4 AD9084s must shared the same 24-lanes available on the VCU118 FMC+ connector. We will also assume RX and TX have the same configuration. If the ADC/DAC configurations are different but the resulting samples rates and late rates are the same this assumption will hold.


## Generate Profile

First from ACE generate we need to generate our desired configuration. Since Triton only has 2 lanes per side we cannot exceed 2 lanes in the profile. The VCU118 will also have a practical lane rate limit of 28 Gbps on the SERDES. Below is a screen show of the configuration used and the exported profiles are provided alongside the example.

![ACE AD9084 Configuration](triton_ace.PNG)

The co-located profile files can be seen below. Only the .bin is used by the driver where the .json will be used for clock modeling and calculations.

In [1]:
!ls id00_triton_M4_L2_RX13p2*

id00_triton_M4_L2_RX13p2.bin	    id00_triton_M4_L2_RX13p2.json
id00_triton_M4_L2_RX13p2.blueprint  id00_triton_M4_L2_RX13p2.log
id00_triton_M4_L2_RX13p2.h	    id00_triton_M4_L2_RX13p2.summary
id00_triton_M4_L2_RX13p2.hpp


# Clock modeling

To correctly configure the FPGA and related clocks we'll use JIF to model the system and calculate the necessary clocks

In [2]:
import adijif
import pprint
import os

vcxo = int(400e6)

# Use generate profile
profile_json = "id00_triton_M4_L2_RX13p2.json"
profile_bin = profile_json.replace('.json','.bin')

sys = adijif.system("ad9084_rx", "ltc6952", "xilinx", vcxo, solver="CPLEX")

sys.fpga.setup_by_dev_kit_name("vcu118")

# Apply datapath config
sys.converter.apply_profile_settings(profile_json)

# Setup clocks
sys.converter.clocking_option = "direct"
sys.add_pll_inline("adf4382", vcxo, sys.converter)
sys.add_pll_sysref("adf4030", sys.clock, sys.converter, sys.fpga)

# Optimizations
sys.clock.minimize_feedback_dividers = False

print(f"Lane rate: {sys.converter.bit_clock/1e9} Gbps")
print(f"Needed Core clock: {sys.converter.bit_clock/66} MHz")

assert sys.converter.bit_clock == int(13.2e9)
assert sys.converter.M == 4
assert sys.converter.L == 2

cfg = sys.solve()

pprint.pprint(cfg)

Lane rate: 13.2 Gbps
Needed Core clock: 200000000.0 MHz
Getting reference clock for adf4030
Getting reference clock for AD9084_RX
{'clock': {'VCO': 200000000.0,
           'n2': 2,
           'out_dividers': [1, 1, 1, 1],
           'output_clocks': {'AD9084_RX_ref_clk': {'divider': 1,
                                                   'rate': 200000000.0},
                             'adf4030_ref_clk': {'divider': 1,
                                                 'rate': 200000000.0},
                             'vcu118_AD9084_RX_device_clk': {'divider': 1,
                                                             'rate': 200000000.0},
                             'vcu118_AD9084_RX_ref_clk': {'divider': 1,
                                                          'rate': 200000000.0}},
           'r2': 4,
           'vcxo': 400000000},
 'clock_ext_pll_adf4382': {'d': 1,
                           'mode': 'integer',
                           'n': 64,
                           

## Generate HDL design
From the generate configuration we can rebuild the HDL to meet the requirements. There may be further adjustments needed to further constrain the reference clocks. We can create the additional arguments for the make command like so:

In [3]:
mode = "64B66B" if sys.converter.jesd_class == "jesd204c" else "8B10B"
make_cmd = (
    f"JESD_MODE={mode} "
    + f"RX_RATE={sys.converter.bit_clock/1e9:.4f} TX_RATE={sys.converter.bit_clock/1e9:.4f} "
    + f"RX_JESD_M={sys.converter.M} TX_JESD_M={sys.converter.M} "
    + f"RX_JESD_L={sys.converter.L} TX_JESD_L={sys.converter.L} "
    + f"RX_JESD_S={sys.converter.S} TX_JESD_S={sys.converter.S} "
    + f"RX_JESD_NP={sys.converter.Np} TX_JESD_NP={sys.converter.Np} "
)
print("Make command:")
print(make_cmd)

Make command:
JESD_MODE=64B66B RX_RATE=13.2000 TX_RATE=13.2000 RX_JESD_M=4 TX_JESD_M=4 RX_JESD_L=2 TX_JESD_L=2 RX_JESD_S=1 TX_JESD_S=1 RX_JESD_NP=16 TX_JESD_NP=16

# Devicetree Mapping

From the generate structure we can map in the clock configuration to the devicetree. This is primarily the LTC6952.

See the [driver doc](https://github.com/analogdevicesinc/linux/blob/staging/xlnx/main-next-ad9084-dev/docs/drivers/iio/adc/apollo/index.rst) for more API specific details of AD9084.

In [7]:
from importlib.util import find_spec

if find_spec("jinja2"):
    import jinja2
    # Read template
    with open("triton_dt.tmpl", "r") as f:
        template = jinja2.Template(f.read())
        rendered = template.render(cfg=cfg, profile_filename=profile_bin)
    with open("triton_dt.dts", "w") as f:
        f.write(rendered)
else:
    print("jinja2 not installed, install with pip install jinja2")

// SPDX-License-Identifier: GPL-2.0
/*
 * Analog Devices Quad-AD9084 RevB
 * https://wiki.analog.com/resources/tools-software/linux-drivers/iio-mxfe/ad9084
 * https://wiki.analog.com/resources/eval/user-guides/ad9084_fmca_ebz/ad9084_fmca_ebz_hdl
 *
 * hdl_project: <quad_apollo/vcu118>
 * board_revision: <Rev.B>
 *
 * Copyright (C) 2025 Analog Devices Inc.
 */

#define DEVICE_PROFILE_NAME	"id00_triton_M4_L2_RX13p2.json"

#define min(X, Y)  ((X) < (Y) ? (X) : (Y))

#define MHz			1000000

#ifndef FREQ_J1_MHz
#define FREQ_J1_MHz		(400 * MHz)
#endif

#ifndef TX_CORE_CLK_MHz
#define TX_CORE_CLK_MHz		(200 * MHz)
#endif

#ifndef RX_CORE_CLK_MHz
#define RX_CORE_CLK_MHz		(200 * MHz)
#endif

#ifndef FPGA_REFCLK_CLK_MHz
#define FPGA_REFCLK_CLK_MHz	(200 * MHz)
#endif

#ifndef SYSREF_CLK_MHz
#define SYSREF_CLK_MHz		( 0.591715976331361 )
#endif

#define ONCHIP_PLL_LOW_FREQ	(400 * MHz)
#define EXT_VCO_FREQ		(2000 * MHz)

#define ADF4382_DIVIDER		1
#define ADF4382_DIG_DELAY	14
#define ADF4382_ANA_DELAY